### Preform an Identity operation on the relevant layers in ArcPro. I need to do it there since geopandas usually runs out of memory when trying to preform such a complex overlap
- See "Analysis" map
- CFLB (from Jordan) - dissolved 
- THLB (using raster analysis might help here with the percentages. Rasterize the THLB on the grid using the THLB PCT field, then do a "count")
- Protected area (AllProtectedAreas)
- BurnSeverity2023
- Burn2024 (Note - lots of these are "holdover fires". Could report areas that are 2024 and not 2023 in output if desired)
- VRIAgeRange (calculated from VRI. Report the intersection with the burns)
- WHA - Timber Harvest field
- UWR - Timber Harvest field
- Schedule K (Note - check how she wants these filtered - "Approved" (data is old))
- Approved Woodlots

In [ ]:
# Identity with the relevant layers:

# See "Analysis" map
# CFLB (from Jordan) - dissolved 
# THLB (using raster analysis might help here with the percentages. Rasterize the THLB on the grid using the THLB PCT field, then do a "count")
# Protected area (AllProtectedAreas)
# BurnSeverity2023
# Burn2024 (Note - lots of these are "holdover fires". Could report areas that are 2024 and not 2023 in output if desired)
# VRIAgeRange (calculated from VRI. Report the intersection with the burns)
# WHA - Timber Harvest field
# UWR - Timber Harvest field
# Schedule K (Note - check how she wants these filtered - "Approved" (data is old))
# Approved Woodlots


In [1]:
import geopandas as gpd
import pandas as pd
import fiona 
import os
# Create output dataframe

# Read final polygon after union operations
gdb = r'\\spatialfiles2.bcgov\work\FOR\RNI\DPC\General_User_Data\nross\BRFN_NE_LUPCE_Analysis\BRFNLupce.gdb'

layer_name = 'BRFN_AnalysisAreas_V2_Dissolve'

df = gpd.read_file(gdb, layer=layer_name)

# Initial processing - recalc area, change some column values
df['AreaHa'] = df['Shape_Area']/10000
df.loc[df['FIRE_NUMBER'] != '', 'DonnieCreek'] = 1

# calculate AFLB and THLB areas
df['aflb_ha'] = df['AreaHa'] * df['aflb_fact']
df['thlb_ha'] = df['AreaHa'] * df['thlb_fact']
df = df.drop(columns=['FIRE_NUMBER', 'aflb_fact', 'thlb_fact', 'Shape_Length', 'Shape_Area', 'geometry'])

print(df.columns)

Index(['HV1Name', 'HV1Zone_A_B_C', 'HV1ZoneLabel', 'Priority_WMB_NAME',
       'TRAPLINE_AREA_IDENTIFIER', 'Full_WMB_Name', 'FRRA',
       'WHA_TIMBER_HARVEST_CODE', 'FID_Burn2024', 'BURN_SEVERITY_RATING',
       'AgeRange', 'UWR_TIMBER_HARVEST_CODE', 'FID_SchK_Blocks',
       'FID_SchK_Woodlots', 'FID_AllProtectedAreas_redo', 'AreaHa',
       'DonnieCreek', 'aflb_ha', 'thlb_ha'],
      dtype='object')


In [ ]:
# Read the input definition excel sheets which list the output rows and columns.
define_xlsx = r'\\spatialfiles.bcgov\work\srm\nr\OMGSS\OTT\GR_2024_1538\03_ArcGIS_Projects\OutputRowsAndCols.xlsx'

define_rows_df = pd.read_excel(define_xlsx, sheet_name='Rows')
define_columns_df = pd.read_excel(define_xlsx, sheet_name='Columns')

In [5]:
# Prepare the output data frame. This can be exported to an Excel
def createOutputDf(gdf):
    # First define the empty dataframe to be added onto later
    outputdf = pd.DataFrame()

    # Loop over the rows df
    for i, r in define_rows_df.iterrows():
        # create a blank "row dataframe" that we will add all the columns to. This will be appended to the output df
        rdf = pd.DataFrame()
        group = r['GroupField']
        areaSeries = eval(r['Area'])
        # Loop over columns df
        for ic, c in define_columns_df.iterrows():
            # get some variables from the dict so it's easier to read:
            maskSeries = eval(c['Mask'])
            cname = c['Name']
            
            # Create "Column dataframe" using a subset of the one defined in the row dictionary.
            cdf = gdf.loc[areaSeries].loc[maskSeries]
            
            # Choose the "Sum Field". If none is specified it does the Area.
            if pd.isnull(c['sumField']):
                sumField = 'AreaHa'
            else:
                sumField = c['sumField']
            
            # Group the results by designated field
            if pd.isnull(group):
                # If the group field is empty, sum the entire selection and remove all other fields
                cdf = pd.DataFrame([cdf[[sumField]].sum()], index=["All"])
                print(sumField, cdf.iloc[0])
            elif ', ' in group:
                # if the group has multiple fields, then concat them together
                group = group.split(", ")
                try:
                    cdf['merged'] = cdf[group[0]].astype('str') + " - " + cdf[group[1]].astype('str')
                    cdf = cdf[['merged', sumField]].groupby('merged').sum()
                except:
                    raise Exception(f"Cannot group by more than two fields. Remove extra commas from: {group} (row {ic+1} in columns).")
            else:
                # Remove all columns except group field and area, then preform group by and sum operations.
                cdf = cdf[[group, sumField]].groupby(group).sum()
        
            # Add category (from row dictionary)
            cdf['Category'] = r['Category']
            
            # Set index to a multi index of Category and "Polygon" name
            cdf = cdf.set_index(['Category', cdf.index.rename('Polygon')])

            # Rename the area field to the column name for output
            cdf = cdf.rename(columns={'AreaHa': cname})
            
            # If rdf is empty, set it to the "cdf"
            if len(rdf) == 0:
                rdf = cdf
            # otherwise, join the cdf on to the existing "rdf"
            else:
                rdf = rdf.join(cdf, how='outer')
        # Add the rdf to the output dataframe
        outputdf = pd.concat([outputdf, rdf])
    return outputdf

C:\Users\nross\AppData\Local\Temp\ipykernel_11128\663967099.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdf['merged'] = cdf[group[0]].astype('str') + " - " + cdf[group[1]].astype('str')
C:\Users\nross\AppData\Local\Temp\ipykernel_11128\663967099.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdf['merged'] = cdf[group[0]].astype('str') + " - " + cdf[group[1]].astype('str')
C:\Users\nross\AppData\Local\Temp\ipykernel_11128\663967099.py:28: SettingWithCopyWarning: 
A value is trying to be set on

In [6]:
# save as Excel with date
from datetime import datetime
now = datetime.now()
date = now.strftime("%Y-%m-%d")
wks = r'\\spatialfiles2.bcgov\work\FOR\RNI\DPC\General_User_Data\nross\BRFN_NE_LUPCE_Analysis'
outfile = os.path.join(wks, 'Deliverables', f'LUP Analysis Pandas {date}.xlsx')
outputdf = createOutputDf(df)
outputdf.style.map(lambda v: "number-format: #,##0").to_excel(outfile)

In [95]:
# Other analysis:

# Amount of old in "protection"
old_df = df.loc[(df['FID_AllProtectedAreas_redo'] > -1)]
old_df = old_df[['AgeRange', 'AreaHa']]
old_df.groupby(['AgeRange']).sum()

,AreaHa
AgeRange,
,43736.951138
100 - 139,19978.415657
140 - 249,58232.791258
250+,720.116175


In [96]:
# Amount of old in Schedule K 
old_df = df.loc[(df['FID_SchK_Blocks'] > -1) | (df['FID_SchK_Woodlots'] > -1)]
old_df.loc[df['FID_SchK_Blocks'] > -1, 'SchK_Type'] = 'Blocks'
old_df.loc[df['FID_SchK_Woodlots'] > -1, 'SchK_Type'] = 'Woodlots'

old_df = old_df[['AgeRange', 'SchK_Type', 'AreaHa']]
old_df = old_df.groupby(['AgeRange', 'SchK_Type']).sum()
# old_df.reset_index()
old_df.reset_index().pivot(index = 'AgeRange', columns='SchK_Type', values='AreaHa')

C:\Users\nross\AppData\Local\Temp\ipykernel_1908\3276119603.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_df.loc[df['FID_SchK_Blocks'] > -1, 'SchK_Type'] = 'Blocks'


SchK_Type,Blocks,Woodlots
AgeRange,,
,901.225375,1997.583071
100 - 139,2031.435725,1111.478693
140 - 249,1532.414945,1273.906786
250+,64.292193,NaN


In [98]:
# Age required for Trapline and WMB Old Growth Targets
# Trapline requires 33% old
# WMB required 25% old

trap = df.loc[df['TRAPLINE_AREA_IDENTIFIER'] != '']
trap = trap[['AgeRange', 'AreaHa']]
trap['Pct'] = trap['AreaHa'] / trap['AreaHa'].sum()
trap = trap.groupby(['AgeRange']).sum()

WMB = df.loc[(df['Priority_WMB_NAME'] != "") | (df['Full_WMB_Name'] == "Cameron River")]
WMB = WMB[['AgeRange', 'AreaHa']]
WMB['Pct'] = WMB['AreaHa'] / WMB['AreaHa'].sum()
WMB = WMB.groupby(['AgeRange']).sum()

x = trap.merge(WMB, on=['AgeRange'])
x

,AreaHa_x,Pct_x,AreaHa_y,Pct_y
AgeRange,,,,
,429110.801619,0.554015,640812.046467,0.517151
100 - 139,232935.913730,0.300738,398464.263518,0.321570
140 - 249,111335.763028,0.143743,198750.650598,0.160397
250+,1164.327119,0.001503,1093.622379,0.000883


In [99]:
df.to_excel('test.xlsx')